# 3. Deep Learning

### Definición RNN
Usar capa SimpleRNN y la columna demanda como target.

In [107]:
import numpy as np
import pandas as pd

import plotly.express as px

from encoding import encoder

from sklearn.preprocessing import MinMaxScaler

from keras.layers import Input, SimpleRNN, Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam


In [113]:
ruta_archivo = "..\data\processed\DF_DEMANDA_10_25_LIMPIO.csv"
df = pd.read_csv(ruta_archivo)

## Preparación de los datos

#### Filtrar datos
Escogemos las columnas de interés y pasamos MWh a GWh

In [114]:
df_filtrado = df[(df['zona'] == 'nacional') & (df['titulo'] == 'Demanda')].sort_values(by='fecha').reset_index(drop=True)
df_filtrado = df_filtrado[['valor_(MWh)', 'año', 'mes', 'dia', 'dia_semana']].reset_index(drop=True)
df_filtrado['valor_(MWh)'] = df_filtrado['valor_(MWh)'].apply(lambda x:round(x*0.001, 3))
df_filtrado.rename(columns={'valor_(MWh)': 'valor_(GWh)'}, inplace=True)

In [115]:
df_filtrado

,valor_(GWh),año,mes,dia,dia_semana
0,605.986,2011,1,1,sábado
1,641.856,2011,1,2,domingo
2,801.297,2011,1,3,lunes
3,833.253,2011,1,4,martes
4,803.476,2011,1,5,miércoles
...,...,...,...,...,...
5165,723.448,2025,2,21,viernes
5166,638.291,2025,2,22,sábado
5167,584.564,2025,2,23,domingo
5168,701.469,2025,2,24,lunes


### Encoding
Vamos a usar un encoding circular, para que el modelo entienda mejor la estacionalidad de los datos.

Primero, definimos los días que tiene cada mes. Debemos hacerlo así, con un diccionario, ya que si intentamos hacer un groupby para sacar cuántos días tiene cada mes en el histórico (viendo los bisiestos), del mes actual solo cogerá el número de días que hayan pasado (por ejemplo, si tenemos datos hasta el 6 de marzo, escalará los datos de ese mes dividiendo entre 6...)

In [116]:
df_filtrado = encoder(df_filtrado)

In [ ]:
df_filtrado 

,valor_(GWh),año,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,dia_mes_sin,dia_mes_cos
0,0.267688,0.0,-0.974928,-0.222521,0.500000,0.866025,0.201299,9.795299e-01
1,0.349813,0.0,-0.781831,0.623490,0.500000,0.866025,0.394356,9.189578e-01
2,0.714859,0.0,0.000000,1.000000,0.500000,0.866025,0.571268,8.207634e-01
3,0.788023,0.0,0.781831,0.623490,0.500000,0.866025,0.724793,6.889669e-01
4,0.719848,0.0,0.974928,-0.222521,0.500000,0.866025,0.848644,5.289640e-01
...,...,...,...,...,...,...,...,...
5165,0.536621,1.0,-0.433884,-0.900969,0.866025,0.500000,-1.000000,-1.836970e-16
5166,0.341651,1.0,-0.974928,-0.222521,0.866025,0.500000,-0.974928,2.225209e-01
5167,0.218641,1.0,-0.781831,0.623490,0.866025,0.500000,-0.900969,4.338837e-01
5168,0.486299,1.0,0.000000,1.000000,0.866025,0.500000,-0.781831,6.234898e-01


Si lo representamos, sale un círculo de radio 1.

In [117]:
fig = px.line(
    data_frame=df_filtrado[:32],
    x='dia_mes_sin',
    y='dia_mes_cos'
)

fig.show()

### Escalado de datos

In [118]:
scaler = MinMaxScaler()
cols_to_scale = ["valor_(GWh)", "año"]  

df_filtrado[cols_to_scale] = scaler.fit_transform(df_filtrado[cols_to_scale]) 

### Secuencias de entrada y salida
Las redes recurrentes aprenden observando una secuencia con sus características y prediciendo el siguiente valor del target (en este caso, la demanda). Lo que haremos será crear ventanas deslizantes de "loockback" días:
- En la X nos guardamos los datos de los 'loockback' días anteriores.
- En la y intentará predecir el día siguiente.
- Devuelve un array para cada una que podrá entrar a la red neuronal.

In [119]:
def create_sequences(df, target_column, lookback):
    X, y = [], []
    for i in range(len(df) - lookback):
        X.append(df.iloc[i:i+lookback].drop(columns=[target_column]).values) 
        y.append(df.iloc[i+lookback][target_column]) 
    return np.array(X), np.array(y)

# Definir la ventana de tiempo - 14 días
lookback = 14  

X, y = create_sequences(df_filtrado, target_column="valor_(GWh)", lookback=lookback)

print("Forma de X:", X.shape)  # (n_samples, lookback, n_features)
print("Forma de y:", y.shape)  # (n_samples,)

Forma de X: (5156, 14, 7)
Forma de y: (5156,)


In [154]:
X

array([[[ 0.00000000e+00, -9.74927912e-01, -2.22520934e-01, ...,
          8.66025404e-01,  2.01298520e-01,  9.79529941e-01],
        [ 0.00000000e+00, -7.81831482e-01,  6.23489802e-01, ...,
          8.66025404e-01,  3.94355855e-01,  9.18957812e-01],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          8.66025404e-01,  5.71268215e-01,  8.20763441e-01],
        ...,
        [ 0.00000000e+00,  9.74927912e-01, -2.22520934e-01, ...,
          8.66025404e-01,  6.51372483e-01, -7.58758123e-01],
        [ 0.00000000e+00,  4.33883739e-01, -9.00968868e-01, ...,
          8.66025404e-01,  4.85301963e-01, -8.74346616e-01],
        [ 0.00000000e+00, -4.33883739e-01, -9.00968868e-01, ...,
          8.66025404e-01,  2.99363123e-01, -9.54139256e-01]],

       [[ 0.00000000e+00, -7.81831482e-01,  6.23489802e-01, ...,
          8.66025404e-01,  3.94355855e-01,  9.18957812e-01],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          8.66025404e-01,  5.71268215e

### Train/Test
Debe mantener la temporalidad: el 80% de los datos más antiguos irán al train set y el 20% restante al test

In [120]:
train_size = int(len(X) * 0.8)

X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

## Modelo RNN
Generamos la estructura de la red neuronal. 
+ En primer lugar usaremos una RNN simple y veremos la función de pérdida.
+ Luego aplicaremos una

### Definición RNN simple

In [121]:
model = Sequential()

# Capa de Entrada - usamos 14 días y le damos el número de columnas independientes
model.add(Input(shape = (lookback, X.shape[2])))

model = Sequential([
    SimpleRNN(64, activation="relu"),  # Capa recurrente
    Dense(32, activation="relu"),  # Capas ocultas
    Dense(1)  # Capa de salida para predecir la demanda
])

model.compile(optimizer = "adam", loss = "mse")

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_6 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [122]:
history = model.fit(x = X_train,
                    y = y_train,
                    validation_data = (X_val, y_val),
                    epochs = 20,
                    verbose=1)

Epoch 1/20


129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0691 - val_loss: 0.0197
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0110 - val_loss: 0.0119
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0098 - val_loss: 0.0201
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0083 - val_loss: 0.0121
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0074 - val_loss: 0.0160
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0082 - val_loss: 0.0113
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0075 - val_loss: 0.0136
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0069 - val_loss: 0.0098
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0067 - val_loss: 0.0178
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0064 - val_loss: 0.0106
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0060 - val_loss: 0.0081
Epoch 12/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.

In [ ]:
fig = px.line(data_frame=history.history,
        y=['loss', 'val_loss'],
        title='Función de pérdida (loss) basada en el MSE',
        labels={'index': 'Época', 'value': 'Pérdida'},
        )

fig.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig.for_each_trace(lambda t: t.update(name="Pérdida Entrenamiento" if t.name == "loss" else "Pérdida Validación"))

fig.show()

### One-step predictions
Aquí nos cogeremos los datos de los últimos 14 días y con ellos haremos la predicción de los siguientes 14 de uno en uno.

In [142]:
validation_target = y[-14:]
validation_predictions = list()

i = -14

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, i, X.shape[2]))[0, 0]
    i += 1
    
    validation_predictions.append(p)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [143]:
validation_target

array([0.61681663, 0.5647778 , 0.52133846, 0.37236303, 0.25189459,
       0.49076173, 0.54966   , 0.52681274, 0.51160107, 0.5366211 ,
       0.34165121, 0.21864139, 0.48629943, 0.55041326])

In [146]:
# Crear un array con la misma cantidad de columnas que el scaler espera
dummy_features = np.zeros((len(validation_predictions), 1))

# Convertir las predicciones en un array con la forma adecuada
validation_predictions = np.array(validation_predictions).reshape(-1, 1)
validation_target = np.array(validation_target).reshape(-1, 1)

# Unir las predicciones con los valores ficticios
predictions_with_dummy = np.hstack([validation_predictions, dummy_features])
validation_target_dummy = np.hstack([validation_target, dummy_features])

# Aplicar la transformación inversa
predictions_real = scaler.inverse_transform(predictions_with_dummy)[:, 0]  # Solo tomar la columna de interés
validation_real = scaler.inverse_transform(validation_target_dummy)[:, 0]

print(predictions_real)

[758.01296088 772.13619793 782.74004141 702.22039251 629.55464026
 718.70216239 735.97212315 754.1342006  790.55928746 772.57921035
 705.17793063 628.60470312 728.2209808  712.41912837]


In [149]:
fig_one_step = px.line(
                       y=[validation_real, predictions_real],
                       title='Predicción de la demanda en 14 días',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_one_step.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_one_step.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_one_step.show()

### Multi-step prediction
Aquí, también haremos la predicción de los siguientes 14 días, pero aprovechando en cada iteración los nuevos datos aportados por la predicción anterior.